## 自己构建的网络

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_pre
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_pre
from keras.preprocessing import image   
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py
import common
import pickle

Using TensorFlow backend.


## 对图片进行分类

In [2]:
common.divide_images()

In [3]:
models_and_features = {}
models_and_test_features = {}

In [4]:
def batch_extract_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg16_pre)
    models_and_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg19_pre)
    models_and_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), resnet50_pre)
    models_and_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), inceptionV3_pre)
    models_and_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), xception_pre)
    models_and_features['xception'] = features_name

In [5]:
def batch_extract_test_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), vgg16_pre)
    models_and_test_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), vgg19_pre)
    models_and_test_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), resnet50_pre)
    models_and_test_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (299, 299), inceptionV3_pre)
    models_and_test_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (299, 299), xception_pre)
    models_and_test_features['xception'] = features_name

In [6]:
batch_extract_features()

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
250/250 [==============================] - 40s 160ms/step
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
250/250 [==============================] - 48s 191ms/step
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
250/250 [==============================] - 42s 169ms/step
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
250/250 [==============================] - 51s 204ms/step
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
250/250 [==============================] - 84s 337ms/step


In [7]:
batch_extract_test_features()

Found 12500 images belonging to 1 classes.
625/625 [==============================] - 114s 183ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 122s 195ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 107s 171ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 129s 207ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 213s 341ms/step


In [8]:
if len(models_and_features) == 0:
    models_and_features['vgg16'] = 'vgg16_features.npz'
    models_and_features['vgg19'] = 'vgg19_features.npz'
    models_and_features['resnet50'] = 'resnet50_features.npz'
    models_and_features['inceptionV3'] = 'inception_v3_features.npz'
    models_and_features['xception'] = 'xception_features.npz'

In [9]:
if len(models_and_test_features) == 0:
    models_and_test_features['vgg16'] = 'test_vgg16_features.npz'
    models_and_test_features['vgg19'] = 'test_vgg19_features.npz'
    models_and_test_features['resnet50'] = 'test_resnet50_features.npz'
    models_and_test_features['inceptionV3'] = 'test_inception_v3_features.npz'
    models_and_test_features['xception'] = 'test_xception_features.npz'

In [11]:
# 训练+预测
for key, value in models_and_features.items():
    print('Train Model {0}'.format(key))
    features = np.load(value)
    features_test = np.load(models_and_test_features[key])
    train_features = features['train']
    train_labels = features['train_label']
    valid_features = features['valid']
    valid_labels = features['valid_label']
    test_features = features_test['test']
    test_filenames = features_test['test_filename']
    
    model = Sequential()
    model.add(Dense(2048, input_shape=(train_features.shape[1],), activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='{0}.hdf5'.format(key), verbose=1, save_best_only=True)
    train_result = model.fit(train_features, train_labels, epochs=10, batch_size=common.batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])
    
    with open('{0}_history.pkl'.format(key), 'wb') as f:
        pickle.dump(train_result.history, f)
    model.load_weights('{0}.hdf5'.format(key))
    prediction = model.predict(test_features, batch_size=common.batch_size)
    prediction = prediction[:, 0].clip(0.01, 0.99)
    test_fileindex = np.array([os.path.splitext(os.path.split(filename)[1])[0] for filename in test_filenames])
    data = np.stack([test_fileindex, prediction], axis=1)
    tmp = pd.DataFrame(data, columns=['id', 'label'])
    tmp['id'] = tmp['id'].apply(pd.to_numeric)
    submit_frame = pd.read_csv('dogs-vs-cats/sample_submission.csv')
    result = pd.merge(submit_frame, tmp, on="id", how='left')
    result = result.rename(index=str, columns={"label_y": "label"})
    result[['id','label']].to_csv('{0}_predict.csv'.format(key),index=False)
    

Train Model vgg16
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 15s 734us/step - loss: 1.0117 - acc: 0.9325 - val_loss: 0.6427 - val_acc: 0.9596

Epoch 00001: val_loss improved from inf to 0.64273, saving model to vgg16.hdf5
Epoch 2/10
20000/20000 [==============================] - 12s 597us/step - loss: 1.5602 - acc: 0.9023 - val_loss: 0.4897 - val_acc: 0.9692

Epoch 00002: val_loss improved from 0.64273 to 0.48968, saving model to vgg16.hdf5
Epoch 3/10
20000/20000 [==============================] - 12s 597us/step - loss: 2.0916 - acc: 0.8688 - val_loss: 0.5889 - val_acc: 0.9632

Epoch 00003: val_loss did not improve from 0.48968
Epoch 4/10
20000/20000 [==============================] - 12s 596us/step - loss: 1.5013 - acc: 0.9067 - val_loss: 0.9779 - val_acc: 0.9392

Epoch 00004: val_loss did not improve from 0.48968
Epoch 5/10
20000/20000 [==============================] - 12s 596us/step - loss: 2.0072 - acc: 0.8753 - val_l

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 18s 911us/step - loss: 0.2181 - acc: 0.9825 - val_loss: 0.3781 - val_acc: 0.9760

Epoch 00001: val_loss improved from inf to 0.37807, saving model to xception.hdf5
Epoch 2/10
20000/20000 [==============================] - 15s 767us/step - loss: 0.2281 - acc: 0.9852 - val_loss: 0.2346 - val_acc: 0.9852

Epoch 00002: val_loss improved from 0.37807 to 0.23458, saving model to xception.hdf5
Epoch 3/10
20000/20000 [==============================] - 15s 768us/step - loss: 0.3111 - acc: 0.9801 - val_loss: 0.7359 - val_acc: 0.9532

Epoch 00003: val_loss did not improve from 0.23458
Epoch 4/10
20000/20000 [==============================] - 15s 766us/step - loss: 0.2866 - acc: 0.9819 - val_loss: 0.1671 - val_acc: 0.9894

Epoch 00004: val_loss improved from 0.23458 to 0.16713, saving model to xception.hdf5
Epoch 5/10
20000/20000 [==============================] - 15s 767us/step - loss: 0.247